## Problema 2: Você deverá implementar um programa, para ler, tratar e particionar os dados.

O arquivo fonte está disponível em https://st-it-cloud-public.s3.amazonaws.com/people-v2_1E6.csv.gz



### Data Quality

- Higienizar e homogenizar o formato da coluna document
- Detectar através da coluna document se o registro é de uma Pessoa Física ou Pessoa Jurídica, adicionando uma coluna com essa informação
- Higienizar e homogenizar o formato da coluna birthDate

- Existem duas colunas nesse dataset que em alguns registros estão trocadas. Quais são essas colunas?
- Corrigir os dados com as colunas trocadas
- Além desses pontos, existem outras tratamentos para homogenizar esse dataset. Aplique todos que conseguir.


In [1]:
!pip install console-logging  
!pip install fuzzywuzzy
!sudo apt-get install libmysqlclient-dev --yes
!pip install dataset console-logging mysqlclient dataset --no-cache-dir



libmysqlclient-dev is already the newest version (8.0.27-0ubuntu0.20.04.1).


0 upgraded, 0 newly installed, 0 to remove and 4 not upgraded.


In [2]:
from console_logging.console import Console
console = Console()
from datetime import datetime as dt
from Enriquecimento_UF import Logs

## Carregando Base 

In [ ]:
import pandas as pd
import numpy as np
import re

etapa = "Etapa 2 - Data Clear"

console_log = Logs(etapa)

dataset = "https://st-it-cloud-public.s3.amazonaws.com/people-v2_1E6.csv.gz"


console_log.info("Loading dataset :" +dataset)


df = pd.read_csv(dataset,compression='gzip',error_bad_lines=False,sep=';',low_memory=False)

## Removendo caracter especial da coluna Document

In [ ]:
console_log.info("Removento caracter especial")
df['document'] = df['document'].apply(lambda x: re.sub('[^A-Za-z0-9]+', '', x))

In [ ]:
df.head()

## Avaliando Doc para Pessoas Juridica ou Fisica

In [ ]:

console_log.info("Criando Coluna Pessoas, Pessoa fisica x Pessoa Juridica")
df['PESSOA'] = df['document'].apply(lambda document: "JURIDICA" if len(str(document)) == 14 else "FISICA")

## Correção e Parse de Data Hora

In [ ]:
from dateutil.parser import parse

console_log.info("CORREÇÃO DATA HORA")

console.info("INICIO CORREÇÃO DATA HORA")
inicio = dt.now()
df['birthDate'] = df['birthDate'].apply(lambda date: parse (date))

console.info("FIM CORREÇÃO HORA")

console.log("Tempo de Exec: "+str(dt.now() - inicio))

## Processo de correção de phoneNumber e Job Area

In [ ]:
df['phoneNumber'] = df['phoneNumber'].apply(lambda x: re.sub('[^A-Za-z0-9]+', '', x))

In [ ]:
df['jobArea'] = df['jobArea'].apply(lambda x: x.replace("+","").replace("(","").replace(")","").replace("-","").replace(" ",""))

In [ ]:
df.iloc[[20,21,22,42,44,45,46,49,63,70]]

In [ ]:
def fuck_corrige(df):
    aux = df.copy()

    for ids,item in df[['jobArea','phoneNumber']].iterrows():
        if item['jobArea'].isnumeric() == True and item['phoneNumber'].isnumeric() == False:
              aux.loc[ids,"jobArea"] = item["phoneNumber"]
              aux.loc[ids,"phoneNumber"]= item ["jobArea"]

        if item["jobArea"].isnumeric() == False and item['phoneNumber'].isnumeric() == True:
            pass    
        
        if item["jobArea"].isnumeric() == True and item['phoneNumber'].isnumeric() == True:
            aux.loc[ids,"jobArea"] =  ""

        if item["jobArea"].isnumeric() == False and item['phoneNumber'].isnumeric() == False:
            aux.loc[ids,"phoneNumber"]= "" 
    return aux 


In [ ]:

inicio = dt.now()
console_log.info(" TRATAMENTO DE COLUNAS ")

df_aux = fuck_corrige(df)

console.log("Tempo de Exec: "+str(dt.now() - inicio))

In [ ]:
df.iloc[[20,21,22,42,44,45,46,49,63,70]]

In [ ]:
df_aux.iloc[[20,21,22,42,44,45,46,49,63,70]]

## Correção de UF 

In [ ]:
ufs = pd.read_csv("https://raw.githubusercontent.com/mapaslivres/municipios-br/main/tabelas/ufs.csv")

In [ ]:
!pip install fuzzywuzzy

In [ ]:
import string
from fuzzywuzzy import fuzz

etapa = "Etapa 2 - Data Clear"

console_log = Logs(etapa)

console_log.info(" CORRIGINDO UF ")

lista_state = df_aux[['state']].groupby('state').count().index.tolist()

for index, value in ufs[['uf_code','name']].iterrows():
    #
    for uf in lista_state:
        distance = fuzz.ratio(value['name'], string.capwords(uf))  
        if distance > 60:
          print(value['uf_code'], value['name'],uf, distance)
          query = "state == '{}'".format(uf)
          print(query)  
          lista_index_estados = df_aux.query(query).index.to_list()
          df_aux.loc[lista_index_estados,'state'] = value['uf_code']  
          

In [ ]:
df_aux

## Salvando dados 

In [ ]:
!mkdir base_gerada

In [ ]:
console_log.info(" TRATANDO ANO  ")

In [ ]:
df_aux["Ano"] = df_aux["birthDate"].apply(lambda x:str(x).split("-")[0])

In [ ]:
lista_anos = df_aux[['Ano']].groupby('Ano').count().index.tolist()

In [ ]:
[df_aux.query('Ano == "{}"'.format(ano)).to_csv("base_gerada/{}.csv".format(ano),index=False)  for ano in lista_anos ]


### Particionamento de dados tratados com as regras descritas em DATA QUALITY
- Particionar em arquivos PARQUET por estado (state)
- Particionar em arquivos CSV por ano/mes/dia de nascimento (birthDate)

In [ ]:
!pip install fastparquet

In [ ]:
df_aux.to_parquet('base_gerada/df.parquet.gzip',compression='gzip')

In [ ]:
del df
del df_aux

In [ ]:
console_log.info(" FIM ETAPA 2 - NOTEBOOK  ")


### Agregação dos dados Essa resposta estara no (BI) Metabase
- Quais são as 5 PF que mais gastaram (totalSpent)?
- Qual é o valor de gasto médio por estado (state)?
- Qual é o valor de gasto médio por jobArea?
- Qual é a PF que gastou menos (totalSpent)?
- Quantos nomes e documentos repetidos existem nesse dataset?
- Quantas linhas existem nesse dataset?

